# Matrix configuration of interest

Change these parameter values to whatever matrix you're interested in

In [207]:
testList = [
    (3,5),
    (3,10),
    (3,20),
    (5,5),
    (5,10),
    (5,20),
    (10,5),
    (10,10),
    (10,20),
]

tstIdx =0

candidates = testList[tstIdx][0]
voters = testList[tstIdx][1]
tvaType = "ATVA2"

# Load in data

In [ ]:
# Load data
import pickle

import pandas as pd
df : pd.DataFrame

with open(f"tt/testOutput{tvaType}.pkl", "rb") as file:
    df = pickle.load(file)

In [ ]:
df

In [ ]:
df["happiness_measure"] = df["happiness_measure"].replace("Bubble sort distance / Kendall Tau Distance", "Bubble sort")
df["happiness_measure"] = df["happiness_measure"].replace("Get Happiness", "Top-1 binary")
df["risk_measure"] = df["risk_measure"].replace("Naive Probability of Strategic Voting", "Naive PSV")
df["risk_measure"] = df["risk_measure"].replace("Happiness Weighted PSV", "HW PSV")

# Preload df

In [ ]:
# Drop rows on column input, with shapes that are not (candidates, voters)
df = df[df.input.apply(lambda x: x.shape == (candidates, voters))]

In [ ]:
df

# Hapiness

In [ ]:
# Drop columns
df_happiness = df.drop(columns=["risk_measure", "risk_values"])

# Convert the happiness values list to an average value
df_happiness["happiness_values_avg"] = df_happiness["happiness_values"].apply(lambda x: sum(x) / len(x))

# Drop duplicates based on the matrix column
df_happiness["input_str"] = df_happiness["input"].apply(lambda x: str(x))
df_happiness = df_happiness.drop_duplicates(subset=["input_str", "voting_scheme", "happiness_measure"])
df_happiness = df_happiness.drop(columns=["input_str"])

In [ ]:
df_happiness.head()

<h3> Boxplot </h3>
Schema's avg values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#set seaborn plotting aesthetics
sns.set(style='white')

#create grouped bar chart
avgHappinessPlt = sns.barplot(x='happiness_measure', y='happiness_values_avg', hue='voting_scheme', data=df_happiness)
avgHappinessPlt.set(
    title=f'Matrix Size {candidates}x{voters}',
    xlabel='Measurement',
    ylabel='Avg. happiness'
    )

avgHappinessPlt.get_figure().savefig(f"[{tvaType}] avgHappiness ({candidates}c{voters}v).png", transparent=True)

<h3> Boxplot </h3>
Voter specific values

In [ ]:
# Explode df on hapiness values
df_happiness = df_happiness.explode("happiness_values")

In [ ]:
# create 3rd grouped boxplot 
happinessPlot = sns.boxplot(x = df_happiness['happiness_measure'], 
			y = df_happiness['happiness_values'], 
			hue = df_happiness['voting_scheme'], 
			palette = 'husl')
happinessPlot.set(
    title=f'Matrix Size {candidates}x{voters}',
    xlabel='Hapiness measure',
    ylabel='Voters Happiness'
    )

happinessPlot.get_figure().savefig(f"[{tvaType}] Happiness ({candidates}c{voters}v).png", transparent=True)


# Risk

In [ ]:
# create 3rd grouped boxplot 
riskPlot = sns.boxplot(x = df['risk_measure'], 
			y = df['risk_values'], 
			hue = df['voting_scheme'], 
			palette = 'husl')

riskPlot.set(
    title=f'Matrix Size {candidates}x{voters}',
    xlabel='Risk measure',
    ylabel='Risk of voting manipulation'
    )

# Set y-axis limit from 0 to 1
riskPlot.set_ylim(0, 1)

riskPlot.get_figure().savefig(f"[{tvaType}] risk ({candidates}c{voters}v).png", transparent=True)